## ensemble

1. 만드는 이유 : 단일 머신러닝보다 좋은 성능 ( 정확도, 속도 )
2. 다수의 ***약한*** 머신러닝 모델을 종합하여 최종 모델로 사용
3. 다수의 모델구축 -> 학습데이터 분할 / 재사용 필요
4. 단일 모델들의 취합 방법, 연결 방법에 따라 종류 구분

>    - ***Bagging : with replacement, independent***
>
>데이터를 분할에 집중 -> 확률로 따지면 반복가능 (a꺼내고 a를 다시 넣고, a를 다시 꺼낼 수 있다.)
>
>    - ***Boosting : without replacement, sequential***
>
>데이터를 썰어놓고 -> 있는 데이터를 가지고 모델1, 모델2, 모델3 ...을 만들고 각각의 모델을 직렬로 연결하는 것이다.
>
> 각각의 모델에 가중치를 주고 다음 모델을 만들 때 가중치다시 준다. ( 처음에 logistic이 젤 높았다면 다음에 또 유리하게 준다. )
>
>    - ***K-Fold Cross Validation***
>>학습데이터를 k개로 분할
>>
>>검증용 분할 데이터로 순환식 사용
>>
>>다수의 머신러닝 결과를 (통상) 평균하여 결과 도출
>
>    ***- Bootstrapping ( Bagging에서의 데이터 분할 방법 )***
>>학습데이터로 부터 n개의 학습 데이터 세트 구성
>>
>>랜덤 샘플링, with replacement
>>>동일 데이터, 동일 데이터 세트에 중복 포함 가능
>
> **B**ootrap **Agg**regation == (Bagging)
>
> Boostrap 적용하여 구성한 모델의 취합
>
> Overfit 회피 : 과적합 회피
>
> Weak(약한, 정확도 낮은) 모델 여러개 개발, 취합
>
> Out Of Bag(OOB) : 트레이닝 데이터에는 있지만, 트레이닝 세트에 포함되어 있지 않은 데이터들 
>
>    ***- Boosting***
>>다수의 학습데이터 세트 구성, without replacment ( 똑같은 학습데이터 반복 X )
>>
>> - 순차적 적용
>>
>>모델 1 개발, 개발 결과를 모델 2 개발 입력값 활용
>>> 정확도 낮은 feature에 대해 가중치 증가
>>>
>>> 증가된 가중치 -> 적극적인 학습유도 (AdaBoost, Adaptive Boosting{ == 적응형})
> - ***Gradient Tree Boosting***
>>순차적 모델 개발 과정에서의 가중치 조절에 Gradient Decent 사용
> - ***Histogram-Based Gradieny Boositng***
>>분포도 사용, 예측값 구간화 -> **속도향상**
>
> ***- Voting***
>
> 다수의 개발 모델의 취합방법
>> 1. 개별 모델의 예측값을 기초로 **투표** 실시.
>>
>최종 예측값 결정
>>
>> 2. 개별 모델/ 예측값 별 다른 가중치
>
>
> ***- Stacked***
>>다수의 개발 모델의 취합방법
>>>개별 모델의 예측값 축적
>>>최종적용 모델의 입력값으로 사용
>>
>> 다양한 종류의 개별 모델 stacking 가능

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay, roc_auc_score

In [19]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data', header=None)
data.head()

X = data.iloc[:, 1:8]
y = data.iloc[:, 0].astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 10)

In [15]:
m = RandomForestClassifier(n_estimators = 150, max_depth = 3)
m.fit(X_train, y_train)
print('Train score : ', m.score(X_train, y_train))
print('Test score : ', m.score(X_test, y_test))

Train score :  0.558109833971903
Test score :  0.5473684210526316


In [9]:
print(m.estimator_)
print(len(m.estimators_))

DecisionTreeClassifier()
100


In [22]:
classifiers = [LogisticRegression(solver = 'liblinear'), GaussianNB(), DecisionTreeClassifier(max_depth = 7)]
classifier_names = ['logistic_reg', 'NaiveBayes', 'DecisionTree']
est = []

for n, c in zip(classifier_names, classifiers):
    est.append((n, c))
print(est)
    
m = VotingClassifier(est)
m.fit(X_train, y_train)

print('Train score : ', m.score(X_train, y_train))
print('Test score : ', m.score(X_test, y_test))

[('logistic_reg', LogisticRegression(solver='liblinear')), ('NaiveBayes', GaussianNB()), ('DecisionTree', DecisionTreeClassifier(max_depth=7))]
Train score :  0.5743933588761175
Test score :  0.5320574162679426
